In [1]:
import numpy as np
import pandas as pd
from googleapiclient.discovery import build

In [2]:
# Youtube Data API:
yt = build(
    serviceName = "youtube",
    version = "v3",
    developerKey = "" # my key
)

In [5]:
### Channels data:

# Channels names:
channel_title = ["Tom Scott"]

# Channels Ids (from content = "https://www.youtube.com/channel/):
channel_id = ["UCBa659QWEk1AI4Tg--mrJ2A"]

# Extract the videos data for the channels:
content = yt.channels().list(id = channel_id,
                             part = "contentDe").execute()
sa

In [10]:
content = yt.channels().list(id = channel_id,
                             part = "contentDe").execute()

In [11]:
content

{'kind': 'youtube#channelListResponse',
 'etag': 'PtCpAhUpINHxGRGeYqwEPUPTFmM',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': 'oqcIn8VyKliVdTlrKEkyvZVd7K0',
   'id': 'UCXvA7wLxIJDa1SWYWl7e-oQ',
   'contentDetails': {'relatedPlaylists': {'likes': '',
     'uploads': 'UUXvA7wLxIJDa1SWYWl7e-oQ'}}}]}

In [12]:
uploadId = content["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]
all_videos = []
next_pg_token = None
res = yt.playlistItems().list(playlistId = uploadId,
                              maxResults = 50,
                              part = "snippet",
                              pageToken = next_pg_token).execute()
while True:
    res = yt.playlistItems().list(playlistId = uploadId,
                                  maxResults = 50,
                                  part = "snippet",
                                  pageToken = next_pg_token).execute()
    all_videos += res["items"]
    next_pg_token = res.get("nextPageToken")
    if next_pg_token is None:
        break


In [13]:
all_videos

[{'kind': 'youtube#playlistItem',
  'etag': 'tXRFas022mY5us8XJii3cz09ZvA',
  'id': 'VVVYdkE3d0x4SUpEYTFTV1lXbDdlLW9RLm00dGZudHU0WFM4',
  'snippet': {'publishedAt': '2021-07-12T19:42:09Z',
   'channelId': 'UCXvA7wLxIJDa1SWYWl7e-oQ',
   'title': 'Around the world with The Tim Traveller International Staycation Challenge',
   'description': 'We wanted to take part in The Tim Traveller Staycation Challenge, so we traveled around the world in Brussels. :)\nhttps://www.youtube.com/watch?v=V6LpWGfkhHU\n\n#internationalstaycationchallenge',
   'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/m4tfntu4XS8/default.jpg',
     'width': 120,
     'height': 90},
    'medium': {'url': 'https://i.ytimg.com/vi/m4tfntu4XS8/mqdefault.jpg',
     'width': 320,
     'height': 180},
    'high': {'url': 'https://i.ytimg.com/vi/m4tfntu4XS8/hqdefault.jpg',
     'width': 480,
     'height': 360},
    'standard': {'url': 'https://i.ytimg.com/vi/m4tfntu4XS8/sddefault.jpg',
     'width': 640,
     'height':

In [14]:
videos_ids = list(map(lambda x: x["snippet"]["resourceId"]["videoId"], all_videos))

In [15]:
videos_ids

['m4tfntu4XS8', 'N1dEc6e7Smk', 'Q31ChdEFkYk', 'GSU9zYBZPy8']

In [16]:
stats = []
for i in range(0, len(videos_ids), 40):
    res = (yt).videos().list(id = ",".join(videos_ids[i:i + 40]),
                             part = "statistics").execute()
    stats += res["items"]

In [17]:
stats

[{'kind': 'youtube#video',
  'etag': 'HF3swoWAvihifsUVJz8RNqi6VUE',
  'id': 'm4tfntu4XS8',
  'statistics': {'viewCount': '3584',
   'likeCount': '555',
   'dislikeCount': '1',
   'favoriteCount': '0',
   'commentCount': '53'}},
 {'kind': 'youtube#video',
  'etag': 'QZZUdQRzIGMRspPVPAVo0anMEOg',
  'id': 'N1dEc6e7Smk',
  'statistics': {'viewCount': '900',
   'likeCount': '188',
   'dislikeCount': '0',
   'favoriteCount': '0',
   'commentCount': '20'}},
 {'kind': 'youtube#video',
  'etag': 'F_znARqxvQCts8FlMi7BW8a1s6s',
  'id': 'Q31ChdEFkYk',
  'statistics': {'viewCount': '953',
   'likeCount': '146',
   'dislikeCount': '0',
   'favoriteCount': '0',
   'commentCount': '21'}},
 {'kind': 'youtube#video',
  'etag': '_u-R-6Y7XeCgXQ7BM1HLZbTbRH0',
  'id': 'GSU9zYBZPy8',
  'statistics': {'viewCount': '2369',
   'likeCount': '293',
   'dislikeCount': '0',
   'favoriteCount': '0',
   'commentCount': '53'}}]

In [18]:
video_id, views, likes, dislikes, favorites, comments = [], [], [], [], [], []
for i in range(len(stats)):
    video_id += [stats[i]["id"]]
    views += [stats[i]["statistics"]["viewCount"]]
    likes += [stats[i]["statistics"]["likeCount"]]
    dislikes += [stats[i]["statistics"]["dislikeCount"]]
    favorites += [stats[i]["statistics"]["favoriteCount"]]
    comments += [stats[i]["statistics"]["commentCount"]]
df_videos = pd.DataFrame(
    {
        "channel_title": channel_title,
        "channel_id": channel_id,
        "video_title": list(map(lambda x: x["snippet"]["title"], all_videos)),
        "video_id": video_id,
        "views": views,
        "likes": likes,
        "dislikes": dislikes,
        "favorites": favorites,
        "comments": comments        
    }
)

In [19]:
df_videos

,channel_title,channel_id,video_title,video_id,views,likes,dislikes,favorites,comments
0,arewethereyet,UCXvA7wLxIJDa1SWYWl7e-oQ,Around the world with The Tim Traveller Intern...,m4tfntu4XS8,3584,555,1,0,53
1,arewethereyet,UCXvA7wLxIJDa1SWYWl7e-oQ,How the university of Leuven kept losing its l...,N1dEc6e7Smk,900,188,0,0,20
2,arewethereyet,UCXvA7wLxIJDa1SWYWl7e-oQ,The very lively ghost town of Doel.,Q31ChdEFkYk,953,146,0,0,21
3,arewethereyet,UCXvA7wLxIJDa1SWYWl7e-oQ,"How Brussels lost it's river, and where you ca...",GSU9zYBZPy8,2369,293,0,0,53
